In [1]:
from erddapy import ERDDAP
from erddapy import utilities

In [2]:
import json
import urllib
import requests
file = urllib.request.urlopen('https://raw.githubusercontent.com/IrishMarineInstitute/search-erddaps/master/erddaps.json')
servers = json.loads(file.read())
file.getcode()

200

In [3]:
servers=servers[:3]

In [4]:
print(servers)

[{'url': 'https://coastwatch.pfeg.noaa.gov/erddap/'}, {'url': 'https://apdrc.soest.hawaii.edu/erddap/'}, {'url': 'https://erddap.bco-dmo.org/erddap/'}]


In [26]:
time_min = '2010-07-10T00:00:00Z'
time_max = '2016-08-10T00:00:00Z'
bbox = [-72.0, -69, 38, 41]

kw = {
   'search_for': 'all',
   'min_lon': bbox[0],
   'max_lon': bbox[1],
   'min_lat': bbox[2],
   'max_lat': bbox[3],
   'min_time': time_min,
   'max_time': time_max,
}

In [55]:
import pandas as pd
all_datasets=pd.DataFrame()

for key in servers:
    url = key['url']
    url = url.rstrip("/")
    r = requests.get(url)
    try:
        r.raise_for_status()
        e = ERDDAP(
                 server=url,
                 protocol='tabledap',
                 response='csv'
           )
        print(e.get_search_url(**kw))
    except requests.exceptions.RequestException as err:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.HTTPError as errh:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.ConnectionError as errc:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.Timeout as errt:
        print("Bad ERDDAP!!! {}".format(url))
    datasets = pd.read_csv('%s'%e.get_search_url(**kw))
    datasets['server'] = url
    all_datasets = pd.concat([all_datasets,datasets])

https://coastwatch.pfeg.noaa.gov/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=(ANY)&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1278720000.0&maxTime=1470787200.0&searchFor=all
https://apdrc.soest.hawaii.edu/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=(ANY)&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1278720000.0&maxTime=1470787200.0&searchFor=all
https://erddap.bco-dmo.org/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=(ANY)&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1278720000.0&maxTime=1470787200.0&searchFor=all


/anaconda3/envs/erddap/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [76]:
tabledap = all_datasets.loc[all_datasets['tabledap'].notnull(),'Info']
ii=0
for csv_url in tabledap:
    
    meta = pd.read_csv(str(csv_url))
    print(meta['Variable Name'].unique())
    print()
    e = ERDDAP(
        server=all_datasets['server'].iloc[ii],
        protocol='tabledap')

    e.dataset_id = all_datasets['Dataset ID'].unique()
    e.protocol = "tabledap"
    ii+=1
    

['NC_GLOBAL' 'trajectory' 'institution' 'platform_id' 'platform_type'
 'wmo_id' 'ctd_make_model' 'segment_id' 'time_uv' 'lat_uv' 'lon_uv' 'u'
 'v' 'uv_qc' 'profile_id' 'profile_qc' 'time' 'latitude' 'longitude'
 'depth' 'd_qc' 'pressure' 'p_qc' 'temperature' 't_qc' 'salinity' 's_qc'
 'conductivity' 'c_qc' 'precise_time' 'precise_time_qc' 'precise_lat'
 'precise_lat_qc' 'precise_lon' 'precise_lon_qc']

['NC_GLOBAL' 'trajectory' 'org' 'type' 'platform' 'cruise' 'station_id'
 'longitude' 'latitude' 'time' 'depth' 'temperature' 'salinity']

['NC_GLOBAL' 'longitude' 'latitude' 'time' 'ob_clm' 'ob_csgm' 'ob_glb'
 'ob_gsgm' 'ob_qc' 'ob_rcpt' 'ob_rgn' 'ob_rsgm' 'ob_sign' 'ob_sst'
 'ob_typ' 'ob_wm']

['NC_GLOBAL' 'vehicleName' 'ais' 'feed_version' 'time' 'latitude'
 'longitude' 'MMSI' 'COG' 'SOG' 'ROT' 'Heading' 'NavigationStatus']

['NC_GLOBAL' 'vehicleName' 'board_id' 'task_id' 'time' 'latitude'
 'longitude' 'wave_height' 'average_wave_period' 'dominant_wave_period'
 'wave_direction' 'number_

In [75]:
all_datasets['server'][ii]


0    https://coastwatch.pfeg.noaa.gov/erddap
0      https://apdrc.soest.hawaii.edu/erddap
0          https://erddap.bco-dmo.org/erddap
Name: server, dtype: object

In [51]:
meta['Variable Name'].unique()

array(['NC_GLOBAL', 'cruise_id', 'project', 'station_GEOTRC', 'sta_PI',
       'latitude', 'longitude', 'cast_GEOTRC', 'event_GEOTRC', 'depth',
       'depth_GEOTRC_CTD_round', 'sample_GEOTRC', 'sample_bottle_GEOTRC',
       'bottle_GEOTRC', 'grid_type', 'grid_num', 'SXRF_run', 'mda_id',
       'cell_type', 'cell_vol', 'cell_C', 'cell_Si', 'cell_P', 'cell_S',
       'cell_Mn', 'cell_Fe', 'cell_Co', 'cell_Ni', 'cell_Cu', 'cell_Zn',
       'light_image_filename', 'chl_image_filename', 'SXRF_map_filename',
       'SXRF_spectrum_filename', 'time'], dtype=object)

In [8]:
# from erddapy import ERDDAP


# e = ERDDAP(
#   server='https://data.ioos.us/gliders/erddap',
#   protocol='tabledap',
# )

# e.response = 'csv'
# e.dataset_id = 'whoi_406-20160902T1700'
# e.constraints = {
#     'time>=': '2016-07-10T00:00:00Z',
#     'time<=': '2017-02-10T00:00:00Z',
#     'latitude>=': 38.0,
#     'latitude<=': 41.0,
#     'longitude>=': -72.0,
#     'longitude<=': -69.0,
# }
# e.variables = [
#     'depth',
#     'latitude',
#     'longitude',
#     'salinity',
#     'temperature',
#     'time',
# ]

In [9]:
    if url != 'http://erddap.emodnet-physics.eu/erddap' and \
    url != 'https://erddap.marine.ie/erddap' and \
    url != 'http://oos.soest.hawaii.edu/erddap' and \
    url != 'http://erddap.secoora.org/erddap' and \
    url != 'https://ecowatch.ncddc.noaa.gov/erddap' and \
    url != 'http://dap.onc.uvic.ca/erddap':

SyntaxError: unexpected EOF while parsing (<ipython-input-9-a1f1d5c2bc5d>, line 6)